# TruLens-Canopy Quickstart

 Canopy is an open-source framework and context engine built on top of the Pinecone vector database so you can build and host your own production-ready chat assistant at any scale. By integrating TruLens into your Canopy assistant, you can quickly iterate on and gain confidence in the quality of your chat assistant.

 [![Open In
Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/trulens_eval/examples/expositional/frameworks/canopy/canopy_quickstart.ipynb)

In [1]:
# !pip install -qU canopy-sdk trulens-eval cohere ipywidgets

In [2]:
import numpy

assert (
    numpy.__version__ >= "1.26"
), "Numpy version did not updated, if you are working on Colab please restart the session."

## Set Keys

In [3]:
import os

os.environ["PINECONE_API_KEY"] = (
    "YOUR_PINECONE_API_KEY"  # take free trial key from https://app.pinecone.io/
)
os.environ["OPENAI_API_KEY"] = (
    "YOUR_OPENAI_API_KEY"  # take free trial key from https://platform.openai.com/api-keys
)
os.environ["CO_API_KEY"] = (
    "YOUR_COHERE_API_KEY"  # take free trial key from https://dashboard.cohere.com/api-keys
)

In [4]:
assert (
    os.environ["PINECONE_API_KEY"] != "YOUR_PINECONE_API_KEY"
), "please provide PINECONE API key"
assert (
    os.environ["OPENAI_API_KEY"] != "YOUR_OPENAI_API_KEY"
), "please provide OpenAI API key"
assert (
    os.environ["CO_API_KEY"] != "YOUR_COHERE_API_KEY"
), "please provide Cohere API key"

In [5]:
from pinecone import PodSpec

# Defines the cloud and region where the index should be deployed
# Read more about it here - https://docs.pinecone.io/docs/create-an-index
spec = PodSpec(environment="gcp-starter")

## Load data
Downloading Pinecone's documentation as data to ingest to our Canopy chatbot:

In [6]:
import warnings

import pandas as pd

warnings.filterwarnings("ignore")

data = pd.read_parquet(
    "https://storage.googleapis.com/pinecone-datasets-dev/pinecone_docs_ada-002/raw/file1.parquet"
)
data.head()

,id,text,source,metadata
0,728aeea1-1dcf-5d0a-91f2-ecccd4dd4272,# Scale indexes\n\n[Suggest Edits](/edit/scali...,https://docs.pinecone.io/docs/scaling-indexes,"{'created_at': '2023_10_25', 'title': 'scaling..."
1,2f19f269-171f-5556-93f3-a2d7eabbe50f,# Understanding organizations\n\n[Suggest Edit...,https://docs.pinecone.io/docs/organizations,"{'created_at': '2023_10_25', 'title': 'organiz..."
2,b2a71cb3-5148-5090-86d5-7f4156edd7cf,# Manage datasets\n\n[Suggest Edits](/edit/dat...,https://docs.pinecone.io/docs/datasets,"{'created_at': '2023_10_25', 'title': 'datasets'}"
3,1dafe68a-2e78-57f7-a97a-93e043462196,# Architecture\n\n[Suggest Edits](/edit/archit...,https://docs.pinecone.io/docs/architecture,"{'created_at': '2023_10_25', 'title': 'archite..."
4,8b07b24d-4ec2-58a1-ac91-c8e6267b9ffd,# Moving to production\n\n[Suggest Edits](/edi...,https://docs.pinecone.io/docs/moving-to-produc...,"{'created_at': '2023_10_25', 'title': 'moving-..."


In [7]:
print(
    data["text"][50][:847]
    .replace("\n\n", "\n")
    .replace("[Suggest Edits](/edit/limits)", "")
    + "\n......"
)
print("source: ", data["source"][50])

# Limits
This is a summary of current Pinecone limitations. For many of these, there is a workaround or we're working on increasing the limits.

## Upserts

Max vector dimensionality is 20,000.

Max size for an upsert request is 2MB. Recommended upsert limit is 100 vectors per request.

Vectors may not be visible to queries immediately after upserting. You can check if the vectors were indexed by looking at the total with `describe_index_stats()`, although this method may not work if the index has multiple replicas. Pinecone is eventually consistent.

Pinecone supports sparse vector values of sizes up to 1000 non-zero values.

## Queries

Max value for `top_k`, the number of results to return, is 10,000. Max value for `top_k` for queries with `include_metadata=True` or `include_data=True` is 1,000.

......
source:  https://docs.pinecone.io/docs/limits


## Setup Tokenizer

In [8]:
from canopy.tokenizer import Tokenizer

Tokenizer.initialize()

tokenizer = Tokenizer()

tokenizer.tokenize("Hello world!")

['Hello', ' world', '!']

## Create and Load Index

In [9]:
from canopy.knowledge_base import KnowledgeBase
from canopy.knowledge_base import list_canopy_indexes
from canopy.models.data_models import Document
from tqdm.auto import tqdm

index_name = "pinecone-docs"

kb = KnowledgeBase(index_name)

if not any(name.endswith(index_name) for name in list_canopy_indexes()):
    kb.create_canopy_index(spec=spec)

kb.connect()

documents = [Document(**row) for _, row in data.iterrows()]

batch_size = 100

for i in tqdm(range(0, len(documents), batch_size)):
    kb.upsert(documents[i : i + batch_size])

  0%|          | 0/1 [00:00<?, ?it/s]

## Create context and chat engine

In [10]:
from canopy.context_engine import ContextEngine

context_engine = ContextEngine(kb)

from canopy.chat_engine import ChatEngine

chat_engine = ChatEngine(context_engine)

API for chat is exactly the same as for OpenAI:

In [11]:
from canopy.models.data_models import UserMessage

chat_history = [
    UserMessage(
        content="What is the the maximum top-k for a query to Pinecone?"
    )
]

chat_engine.chat(chat_history).choices[0].message.content

'The maximum value for `top_k` in a Pinecone query is 10,000. However, when the query includes metadata or data, the maximum value for `top_k` is limited to 1,000.\n(Source: https://docs.pinecone.io/docs/limits)'

## Instrument static methods used by engine with TruLens 

In [ ]:
warnings.filterwarnings("ignore")
from canopy.context_engine import ContextEngine
from trulens.core.app.custom import instrument

instrument.method(ContextEngine, "query")

from canopy.chat_engine import ChatEngine

instrument.method(ChatEngine, "chat")

## Create feedback functions using instrumented methods

In [13]:
from trulens.core import Tru

tru = Tru(database_redact_keys=True)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🔒 Secret keys will not be included in the database.


In [14]:
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.ext.provider.openai import OpenAI as fOpenAI

# Initialize provider class
provider = fOpenAI()

grounded = Groundedness(groundedness_provider=provider)

intput = Select.RecordCalls.chat.args.messages[0].content
context = (
    Select.RecordCalls.context_engine.query.rets.content.root[:]
    .snippets[:]
    .text
)
output = Select.RecordCalls.chat.rets.choices[0].message.content

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons,
        name="Groundedness",
        higher_is_better=True,
    )
    .on(context.collect())
    .on(output)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(
        provider.relevance_with_cot_reasons,
        name="Answer Relevance",
        higher_is_better=True,
    )
    .on(intput)
    .on(output)
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons,
        name="Context Relevance",
        higher_is_better=True,
    )
    .on(intput)
    .on(context)
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.context_engine.query.rets.content.root[:].snippets[:].text.collect() .
✅ In Groundedness, input statement will be set to __record__.app.chat.rets.choices[0].message.content .
✅ In Answer Relevance, input prompt will be set to __record__.app.chat.args.messages[0].content .
✅ In Answer Relevance, input response will be set to __record__.app.chat.rets.choices[0].message.content .
✅ In Context Relevance, input question will be set to __record__.app.chat.args.messages[0].content .
✅ In Context Relevance, input statement will be set to __record__.app.context_engine.query.rets.content.root[:].snippets[:].text .


## Create recorded app and run it

In [15]:
from trulens.core import TruCustomApp

app_id = "canopy default"
tru_recorder = TruCustomApp(
    chat_engine,
    app_id=app_id,
    feedbacks=[f_groundedness, f_qa_relevance, f_context_relevance],
)

In [16]:
from canopy.models.data_models import UserMessage

queries = [
    [
        UserMessage(
            content="What is the maximum dimension for a dense vector in Pinecone?"
        )
    ],
    [UserMessage(content="How can you get started with Pinecone and TruLens?")],
    [
        UserMessage(
            content="What is the the maximum top-k for a query to Pinecone?"
        )
    ],
]

answers = []

for query in queries:
    with tru_recorder as recording:
        response = chat_engine.chat(query)
        answers.append(response.choices[0].message.content)

As you can see, we got the wrong answer, the limits for sparse vectors instead of dense vectors:

In [17]:
print(queries[0][0].content + "\n")
print(answers[0])

What is the maximum dimension for a dense vector in Pinecone?

The maximum dimension for a dense vector in Pinecone is 4 billion dimensions.


In [25]:
tru.get_leaderboard(app_ids=[app_id])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
canopy default,0.333333,0.769524,0.966667,3.333333,0.002687


## Run Canopy with Cohere reranker

In [19]:
from canopy.knowledge_base.reranker.cohere import CohereReranker

kb = KnowledgeBase(
    index_name=index_name, reranker=CohereReranker(top_n=3), default_top_k=30
)
kb.connect()

reranker_chat_engine = ChatEngine(ContextEngine(kb))

In [20]:
reranking_app_id = "canopy_reranking"
reranking_tru_recorder = TruCustomApp(
    reranker_chat_engine,
    app_id=reranking_app_id,
    feedbacks=[f_groundedness, f_qa_relevance, f_context_relevance],
)

answers = []

for query in queries:
    with reranking_tru_recorder as recording:
        answers.append(
            reranker_chat_engine.chat(query).choices[0].message.content
        )

With reranking we get the right answer!

In [21]:
print(queries[0][0].content + "\n")
print(answers[0])

What is the maximum dimension for a dense vector in Pinecone?

The maximum dimension for a dense vector in Pinecone is 20,000. (Source: https://docs.pinecone.io/docs/limits)


## Evaluate the effect of reranking 

In [24]:
tru.get_leaderboard(app_ids=[app_id, reranking_app_id])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
canopy_reranking,0.833333,0.775000,0.900000,3.333333,0.002118
canopy default,0.333333,0.764286,0.966667,3.333333,0.002687


## Explore more in the TruLens dashboard

In [23]:
from trulens.dashboard import run_dashboard

run_dashboard(tru)

# stop_dashboard(tru) # stop if needed

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.157:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>